In [1]:
# Author Emmanuel Sedicol
import cv2
import import_ipynb
import imutils
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import POSITION_EXTRACTION as pex

from IPython.display import clear_output
from imutils.video import VideoStream
from imutils.video import FPS
from skimage import measure
from YOLO import yolo

DEBUG = False

importing Jupyter notebook from POSITION_EXTRACTION.ipynb


# Inner Functions

In [2]:
# transform input frame to the corect structure before being classified my the model
def predict_object(frame):
    model = tf.keras.models.load_model("MODEL")
    CATEGORIES = ['basketball', 'hoop']
    
    img = cv2.resize(cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY), (80,80))
    img = img.reshape(-1, 80, 80, 1)
    
    # predict image
    prediction = model.predict([img])
    prediction_label = CATEGORIES[int(prediction[0][0])]
    return(prediction_label)

# load image to compare to frame from shot classification
def image_to_compare():
    image_to_compare = cv2.imread("/Users/esedicol/Desktop/BasketballShotDetectection/images/compare_image.png")
    image_to_compare = cv2.resize(image_to_compare, (600,400))
    image_to_compare = cv2.cvtColor(image_to_compare, cv2.COLOR_BGR2GRAY)
    image_to_compare = image_to_compare[int(80):int(130),int(280):int(325)] 

    return image_to_compare

# Using the Structural Similarity Index to see if image of a ball going in the hoop is found in the video
def compare_images(imageA, imageB):
    ssim = measure.compare_ssim(imageA, imageB)
    return ssim

# Ball detection using Opencv
- uses hsv color range to create a mask
- classify detected objects using the function predict_object()

In [3]:
def opencv_ball_detection(frame, bol):
    detected_object = frame.copy()
    
    # frame filtering and image processing
    blur = cv2.GaussianBlur(frame, (15, 15),0)
    erode = cv2.erode(blur, None, iterations=2)
    dilate = cv2.dilate(erode, None, iterations=2)
    hsv = cv2.cvtColor(dilate, cv2.COLOR_BGR2HSV)
    
    # create mask using lower and upper orange range
    mask = cv2.inRange(hsv, (1, 80, 110), (8,200,175))

    # find contours in the mask and initialize the current
    cnts = cv2.findContours(mask.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    
    if  len(cnts) > 0:   
        
        # get maximum contour based on its area value
        c = max(cnts, key=cv2.contourArea)
        ((x, y), radius) = cv2.minEnclosingCircle(c)
        
        # check if the radius meets size criteria
        if radius > 5 and radius < 20 :
            
            # Retrive the frame of detected ball
            detected_object = detected_object[int(y - 20):int(y + 20),int(x - 20):int(x + 20)] 

            if bol == True:
                # if detected object is classifies as a basketball
                if predict_object(detected_object) == "basketball" :
                    # Draw the circle and centroid on the frame,
                    cv2.circle(frame, (int(x), int(y)), int(radius),(0, 255, 255), 2)
                    cv2.circle(frame, (int(x), int(y)), 1, (0, 0, 255), 2)

                    # Draw rectangle around the minimum enclosed circle and dipslay label of detected object
                    cv2.rectangle(frame, (int(x - 20), int(y - 20)), (int(x + 20), int(y + 20)), (255,0,0), 2)
                    cv2.putText(frame, "BASKETBALL", (int(x - 30), int(y - 25)), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 0), 1 )
            else:
                # if detected object is classifies as a basketball
                if predict_object(detected_object) != "basketball" :
                    # Draw the circle and centroid on the frame,
                    cv2.circle(frame, (int(x), int(y)), int(radius),(0, 255, 255), 2)
                    cv2.circle(frame, (int(x), int(y)), 1, (0, 0, 255), 2)

                    # Draw rectangle around the minimum enclosed circle and dipslay label of detected object
                    cv2.rectangle(frame, (int(x - 20), int(y - 20)), (int(x + 20), int(y + 20)), (255,0,0), 2)
                    cv2.putText(frame, "BASKETBALL", (int(x - 30), int(y - 25)), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 0), 1 )
            
    return x, y, frame

### Variables for main_app parameters

In [4]:
path = "VIDEOS/PLAYER_EXTRACTION_TEST2.mov"

court_image = cv2.imread('/Users/esedicol/Desktop/BasketballShotDetectection/images/court/court_invert.png')
court_image  = cv2.resize(court_image, (600,400))

pts_3D = np.array([(0,235), (70, 236), (230, 235), (370, 232), (530, 230), (600, 227), (300, 360), (200, 315), (405, 310)])
pts_2D = np.array([(0, 0), (63, 0),(226, 0), (370, 0), (535, 0), (600, 0), (300, 216), (226, 160), (370, 160) ])

# Main Function

In [7]:
# Main App Function
def main_app(path, court_image, pts_3D, pts_2D):
    vs = cv2.VideoCapture(path)
    fps = FPS().start()
    
    while True:
        # grab the current frame
        _,frame = vs.read()

        # break out of the application if we have no more frames to read
        if frame is None:
            break
            
        # resize frame so to make its constant size througout
        frame = cv2.resize(frame,(600, 400))
        
        compare_frame = frame[int(80):int(130),int(280):int(325)] 
        compare_frame = cv2.cvtColor(compare_frame, cv2.COLOR_BGR2GRAY)
            
        # detect ball using YOLO
        target_b = "ball"
        frame, detected = yolo.yolo_detection(target_b, frame)
        
        # detect person using YOLO
        target_p = "person"
        frame, detected = yolo.yolo_detection(target_p, frame)

        cim1 = image_to_compare()
        ssim = compare_images(compare_frame, cim1)

        # if ssi scores is over 0.9 (meaning good match)
        if ssim > 0.9:
            print("score")
        
        # extract player position and map into 2D court
        _, _, court_2D = pex.estimate_position(frame, court_image, pts_3D, pts_2D)

        # if YOLO doesnt detect ball use opencv method
        if detected == False:
            _, _, frame = opencv_ball_detection(frame, True)

        # plot results
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 8))
        fig.suptitle('Basketball and Player Detection')
        
        # Column one
        ax1.set_title("Original Frame")
        ax1.imshow(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)) 
        ax1.axis("off")
        
        # Column 2
        ax2.set_title("Mask")
        ax2.imshow(court_2D)
        
        ax2.set_xlim(0, 600)
        ax2.set_ylim(0,400)
        ax2.invert_yaxis()
        
        plt.show()
        clear_output(wait=True)
    
    fps.stop()
    vs.release()
    print("[INFO] elasped time: {:.2f}".format(fps.elapsed()))
    print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))

In [8]:
if DEBUG:
    main_app(path, court_image, pts_3D, pts_2D)

[INFO] elasped time: 77.19
[INFO] approx. FPS: 0.00
